In [ ]:
from feast import Entity, Feature, FeatureView, ValueType, FeatureService, PushSource,Field
from feast.data_format import ParquetFormat, AvroFormat, ProtoFormat
from feast.infra.offline_stores.file_source import FileSource
from feast.repo_config import RegistryConfig, RepoConfig
from feast.infra.online_stores.sqlite import SqliteOnlineStoreConfig
from feast import FeatureStore
from datetime import timedelta, datetime
from google.protobuf.duration_pb2 import Duration


import os
import s3fs
import numpy as np
import pandas as pd


In [3]:
s3_endpoint = "http://localhost:9000"

s3 = s3fs.S3FileSystem(key='giddy',
                       secret='giddy269',
                       client_kwargs={'endpoint_url': s3_endpoint}, use_ssl=False)


In [39]:
from feast.types import String, Float64

user_entity = Entity(
    name="user_id", 
    value_type=ValueType.INT64, 
    description="The ID of the patient"
)



# Declaring the source of the first set of features
f_source1 = FileSource(
    path="Artifacts\Raw_ingested_data\2019-Dec.csv",
    timestamp_field="event_timestamp",
    field_mapping={'event_time':'event_timestamp'}
)

# Defining the first set of features
df1_fv = FeatureView(
    name="df1_feature_view",
    ttl=timedelta(days=3),
    entities=[user_entity],
    schema=[
        Field(name="event_type", dtype=String),
        Field(name="product_id", dtype=String),
        Field(name="category_id", dtype=String),
        Field(name="category_code", dtype=String),
        Field(name="brand", dtype=String),
        Field(name="price", dtype=Float64),
        Field(name='user_session',dtype=String)
        ],    
    source=f_source1,
    online=True
)


repo = RepoConfig(
            registry=f"Artifacts/registry.db",
            project='myfeast',
            provider="local",
            offline_store="file",
            #online_store=SqliteOnlineStoreConfig(),
            use_ssl=False, #True, 
            filesystem=s3,
            is_secure=False, #True,
            validate=False, #True,
)

fs = FeatureStore(config=repo)

fs.apply([df1_fv, user_entity])


<>:13: DeprecationWarning: invalid escape sequence '\R'
c:\Users\giddy\Documents\RECOMMENDATION_SYSTEM\venv\lib\site-packages\feast\repo_config.py:265: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(


In [23]:
features = fs.get_feature_view('df1_feature_view')
print(features)

{
  "spec": {
    "name": "df1_feature_view",
    "entities": [
      "user_id"
    ],
    "features": [
      {
        "name": "event_type",
        "valueType": "STRING"
      },
      {
        "name": "product_id",
        "valueType": "STRING"
      },
      {
        "name": "category_id",
        "valueType": "STRING"
      },
      {
        "name": "category_code",
        "valueType": "STRING"
      },
      {
        "name": "brand",
        "valueType": "STRING"
      },
      {
        "name": "price",
        "valueType": "DOUBLE"
      },
      {
        "name": "user_session",
        "valueType": "STRING"
      }
    ],
    "ttl": "259200s",
    "batchSource": {
      "type": "BATCH_FILE",
      "timestampField": "event_time",
      "fileOptions": {
        "uri": "Artifacts\\Raw_ingested_data\u00819-Dec.csv"
      },
      "dataSourceClassType": "feast.infra.offline_stores.file_source.FileSource",
      "name": "Artifacts\\Raw_ingested_data\u00819-Dec.csv"
    },
   

In [26]:
fs.get_entity('user_id')

Entity(
    name='user_id',
    value_type=<ValueType.INT64: 4>,
    join_key='user_id',
    description='The ID of the patient',
    tags={},
    owner='',
    created_timestamp=datetime.datetime(2025, 1, 4, 12, 54, 55, 900961),
    last_updated_timestamp=datetime.datetime(2025, 1, 4, 12, 54, 55, 900961)
)

In [ ]:
import pandas as pd

# Define the entity DataFrame with cleaned timestamps
entity_df = pd.DataFrame.from_dict({
    "user_id": [576802932, 412120092],
    "event_time": ["2019-12-01 00:00:00", "2019-12-01 00:02:34"]
})
entity_df.rename(columns={"event_time": "event_timestamp"}, inplace=True)

# Ensure proper conversion to datetime format
entity_df["event_timestamp"] = pd.to_datetime(entity_df["event_timestamp"])


# List the features you want to retrieve
feature_refs = [
    "df1_feature_view:event_type",
    "df1_feature_view:price",
    "df1_feature_view:brand"
]

# Retrieve the historical features
historical_features = fs.get_historical_features(
    entity_df=entity_df,
    features=feature_refs
).to_df()

print(historical_features)


TypeError: unsupported operand type(s) for /: 'NoneType' and 'str'

: 